In [1]:
from src.configuration.set_up_config_device import (
    get_allowed_cpu_count,
    set_up_config_device,
    set_up_device,
)
from src.data_processing.loading import load_iob_data
from src.data_processing.tokenization import create_tokenized_dataset

2025-04-11 11:50:55 - INFO - Logging system initialized
/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-11 11:50:57 - INFO - PyTorch version 2.6.0 available.
2025-04-11 11:50:57 - INFO - Polars version 1.26.0 available.
2025-04-11 11:50:57 - INFO - Duckdb version 1.2.1 available.


In [2]:
device = set_up_device()
cpu_count = get_allowed_cpu_count()
n_process = set_up_config_device(cpu_count)

2025-04-11 11:50:57 - INFO - Using cpu device
2025-04-11 11:50:57 - INFO - Using 128 CPUs
2025-04-11 11:50:57 - INFO - torch set up to use 64 processes


In [3]:
train_sentences, train_labels = load_iob_data("data/raw/train-nl.tsv")

In [4]:
create_tokenized_dataset(
    train_sentences, train_labels, save_path="data/tokenized/train_tokenized.json"
)

Creating json from Arrow format: 100%|██████████| 3/3 [00:00<00:00,  5.09ba/s]

Tokenized dataset saved at: data/tokenized/train_tokenized.json


Dataset({
    features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 2199
})